In [27]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from src.dnn import *
from src.data_encoders import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Post selection non-linearity

In [31]:
def non_linearity(x, weights, shots=1000):
    n = weights.shape[0]
    
    storage = qk.QuantumRegister(n, name="storage")
    clas_reg = qk.ClassicalRegister(2, name="clas_reg")
    registers = [storage, clas_reg]
    circuit = qk.QuantumCircuit(*registers)

    circuit.ry(x, storage[0])
    circuit.barrier()

    for i in range(n - 1):
        circuit.cx(storage[i], storage[i + 1])

    circuit.barrier()

    for i, w in enumerate(weights):
        circuit.ry(2*w, storage[i])

    circuit.barrier()    

    for i in range(n - 1):
        circuit.cx(storage[i], storage[i + 1])

    circuit.measure([storage[0], storage[-1]], clas_reg)
    
    job = qk.execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts(circuit)
    
    states = ["00", "01", "10", "11"]
    
    for state in states:
        if state not in counts:
            counts[state] = 0
        
    return (counts["11"])/(counts["01"] + counts["11"]), (counts["01"] + counts["11"])/shots, circuit


def gradient(x, y, weights, shots=1000):
    n = weights.shape[0]
    gradient = np.zeros(weights.shape)
    
    for i in range(n): 
        A, B, _ = non_linearity(x, weights, shots)
        weights[i] +=np.pi/4
        c, d, _ = non_linearity(x, weights, shots)
        weights[i] -=np.pi/2
        e, f, _ = non_linearity(x, weights, shots)
        weights[i] +=np.pi/4

        C = 0.5*(c-e)
        D = 0.5*(d-f)
        
        gradient = C + 0.5*A*D/B
        
        gradient = (A - y)*gradient

    
    return gradient
    

In [32]:
np.random.seed(42)
backend = Aer.get_backend('qasm_simulator')
n = 4
weights = np.random.uniform(0, np.pi, n)
x = np.array([0, np.pi/2])
y = np.array([0.3, 0.7])


In [33]:
grad = np.zeros(weights.shape)
m = x.shape[0]

for i in range(50):
    y_pred = []
    
    for x_, y_ in zip(x,y):
        grad += gradient(x_, y_, weights, shots=10000)
        y = non_linearity(x_, weights, shots=10000)
        y_pred.append(y[0])
        #print(y[1])
    
    grad = grad/m 
    
    weights += -0.1*grad
    print(y_pred)

[0.5431347301884584, 0.5845014016820185]
[0.5392547457229904, 0.5904234269885239]
[0.5402981597950152, 0.5951248513674198]
[0.5493471356311023, 0.5897590361445784]
[0.5421898838164535, 0.5954653937947494]
[0.552892561983471, 0.5974369406021155]
[0.5505551618237656, 0.6042242400315831]
[0.5353010625737898, 0.5879983890455095]
[0.5478516774573279, 0.5964671246319921]
[0.5410163126393616, 0.5979603843890959]
[0.540537355644591, 0.6003187886033075]
[0.541308190925162, 0.6103146156909598]
[0.5588547189819725, 0.5927251043530113]
[0.5428031644822293, 0.6019203840768154]
[0.5471742450945835, 0.6068548387096774]
[0.5488873106060606, 0.5967741935483871]
[0.5530864197530864, 0.6030671181039634]
[0.5587054098747933, 0.6084084084084084]
[0.5444391634980988, 0.5998402555910544]
[0.5583323463223973, 0.5952333266573202]
[0.5542469664525339, 0.599880263420475]
[0.5593361986628462, 0.6091556459816887]
[0.546370725854829, 0.6021656306396631]
[0.5572700296735905, 0.6102198455139631]
[0.5617831382286863, 

KeyboardInterrupt: 

In [ ]:
y_pred, y_anc, circuit = non_linearity(0.3, weights, shots=10000)
print(y_pred, y_anc)
print(circuit)